In [1]:
import numpy as np
import pandas as pd 

pd.options.display.max_columns = None 
pd.options.display.max_rows = 100 

In [2]:
df_accepted = pd.read_csv('accepted_2007_to_2018Q4.csv') 
df_rejected = pd.read_csv('rejected_2007_to_2018Q4.csv') 

C:\Users\miazh_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# df_accepted = df_raw_accepted.sample(frac=0.1) 
# df_rejected = df_raw_accepted.sample(frac=0.1) 

In [4]:
# df_accepted = pd.read_csv('sample_accepted.csv') 
# df_rejected = pd.read_csv('sample_rejected.csv') 

In [5]:
df_rejected.shape 

(27648741, 9)

In [6]:
df_accepted.shape 

(2260701, 151)

In [ ]:
'''
For applications prior to November 5, 2013 the risk score is the borrower's FICO score. 
For applications after November 5, 2013 the risk score is the borrower's Vantage score.
'''

def data_process(df_accepted,df_rejected,dummy = False):
    
    accepted = df_accepted.copy() 
    rejected = df_rejected.copy() 
    
    # Processing accepted data 
    accepted = accepted[['loan_amnt', 'purpose', 'dti',
#        'zip_code', 
       'addr_state', 'emp_length', 'issue_d',
       'loan_status', 'last_fico_range_high','last_fico_range_low']]
    
    accepted['issue_d'] = pd.to_datetime(accepted['issue_d'])
    accepted['app_year'] = accepted['issue_d'].dt.year 
    accepted['app_month'] = accepted['issue_d'].dt.month 

    accepted['risk_score'] = (accepted['last_fico_range_high'] + accepted['last_fico_range_low'])/2 
    accepted = accepted.drop(columns = ['issue_d','last_fico_range_high','last_fico_range_low']) 

#     if reverse = False:
#         accepted = accepted.replace({'loan_status' : { 'Charged Off' : 'rejected', 'Late (16-30 days)' : 'rejected', 
#                                        'Late (31-120 days)' : 'rejected',
#                                        'Does not meet the credit policy. Status:Charged Off' : 'rejected',
#                                        'Fully Paid' : 'accepted', 'Current' : 'accepted',
#                                        'In Grace Period' : 'accepted',
#                                        'Does not meet the credit policy. Status:Fully Paid' : 'accepted'}})

    accepted['loan_status'] =1
            
    
    # Processing rejected data 
    rejected = rejected[['Amount Requested', 'Application Date', 'Loan Title',
#        'zip_code', 
       'Risk_Score', 'Debt-To-Income Ratio', 'Zip Code',
       'State','Employment Length']]
    
    rejected = rejected.rename(columns={"Amount Requested": "loan_amnt", "Application Date": "app_date", "Loan Title": "purpose",
                             "Risk_Score": "risk_score", "Debt-To-Income Ratio": "dti", "Zip Code": "zip_code",
                             "State": "addr_state", "Employment Length": "emp_length"})
    rejected['loan_status'] = 0
    
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*consolidation.*$)', 'debt_consolidation') 
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*loan.*$)', 'debt_consolidation') 
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*debt.*$)', 'debt_consolidation')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*card.*$)', 'credit_card')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*credit.*$)', 'credit_card')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*house.*$)', 'house') 
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*business.*$)', 'small_business')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*car.*$)', 'car')
    rejected['purpose'] = rejected['purpose'].str.replace(r'(^.*education.*$)', 'education')
    
    
    rejected['purpose'] = np.where(rejected['purpose'].isin(['house', 'credit_card', 'major_purchase', 'debt_consolidation',
       'other', 'moving', 'small_business', 'home_improvement',
       'vacation', 'medical', 'renewable_energy', 'car', 'wedding']), rejected['purpose'], 'other')

    rejected['app_date'] = pd.to_datetime(rejected['app_date'])
    rejected['app_year'] = rejected['app_date'].dt.year
    rejected['app_month'] = rejected['app_date'].dt.month

    rejected = rejected.drop(columns = 'app_date')
    rejected = rejected.drop(columns = 'zip_code')
    
    rejected['dti'] = rejected['dti'].str.replace('%','')
    
    # Merge accepted and rejected data for later process
    df_processed = pd.concat([accepted, rejected], sort=True)
    

    # Convert categorical to numerical-- 10 means more than 10 years 
    df_processed['emp_length'] = df_processed['emp_length'].str.extract('(\d+)')


    # Missing Values: risk_score, emp_length, dti
    df_processed['emp_length'] = df_processed['emp_length'].fillna(0)
    df_processed['emp_length'] = df_processed['emp_length'].astype(int)
    df_processed['risk_score'] = df_processed['risk_score'].fillna(0)
    df_processed['dti'] = df_processed['dti'].fillna(0)
    df_processed['dti'] = df_processed['dti'].astype(float)
    df_processed = df_processed.dropna()
    
    
    df_undummpy = df_processed.copy()
    df_processed = pd.get_dummies(df_processed)
    
    if dummy == False:
        return df_processed

    if dummy == True:
        return df_undummpy
    
df_processed = data_process(df_accepted,df_rejected)

In [ ]:
df = df_processed.groupby(['app_year', 'loan_status']).size().reset_index(name='counts')
df 

In [ ]:
df_accepted['issue_d'] = pd.to_datetime(df_accepted['issue_d'])
df_accepted['app_year'] = df_accepted['issue_d'].dt.year
df_accepted.groupby(['app_year']).size()

In [ ]:
df_accepted.groupby(['app_year']).agg({'funded_amnt': 'sum'}).reset_index()

In [ ]:
df_accepted['emp_length'] = df_accepted['emp_length'].str.extract('(\d+)')

# Missing Values: risk_score, emp_length, dti
df_accepted['emp_length'] = df_accepted['emp_length'].fillna(0)
df_accepted['emp_length'] = df_accepted['emp_length'].astype(int)

In [ ]:
df_accepted.groupby(['app_year']).agg({'emp_length': 'mean'}).reset_index()

In [ ]:
df_accepted['risk_score'] = (df_accepted['last_fico_range_high'] + df_accepted['last_fico_range_low'])/2
df_accepted['risk_score'] = df_accepted['risk_score'].fillna(0)

In [ ]:
df_accepted.groupby(['app_year']).agg({'risk_score': 'mean'}).reset_index()

In [ ]:
df_accepted[df_accepted.loan_status == 'Charged Off'].groupby(['app_year']).size()

In [ ]:
df_rejected['app_date'] = pd.to_datetime(df_rejected['Application Date'])
df_rejected['app_year'] = df_rejected['app_date'].dt.year
df_rejected.groupby(['app_year']).size()

In [ ]:
df_processed.groupby(['loan_status']).agg({'loan_amnt': 'mean'}).reset_index()

In [ ]:
df_rejected.groupby(['Loan Title']).size().reset_index(name='counts').sort_values(by='counts',ascending = False).head(20)

In [ ]:
from sklearn.cluster import KMeans
from sklearn import preprocessing
import matplotlib.pyplot as plt

In [ ]:
df_processed_nostatus = df_processed.drop(['loan_status'], axis = 1)
scaler = preprocessing.StandardScaler()
df_processed_nostatus = scaler.fit_transform(df_processed_nostatus)

In [ ]:
wcss = []
for i in range(1, 10):
    kmeans = KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
    kmeans.fit(df_processed_nostatus)
    wcss.append(kmeans.inertia_)
plt.plot(range(1, 10), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=10, init='k-means++', max_iter=300, n_init=10)
kmeans = kmeans.fit(df_processed_nostatus)
labels = kmeans.predict(df_processed_nostatus)
C = kmeans.cluster_centers_

In [ ]:
df_processed ['label'] = labels.tolist()
df_processed.groupby(['label','loan_status']).size()

In [ ]:
unique_elements = np.unique(labels)
unique_elements

In [ ]:
for x in unique_elements:
    print('label: ', x)
    print(df_processed[df_processed['label']==x].sample(10))
    print('-----------------------------------------------')
    print(df_processed[df_processed['label']==x].describe())
    print('-----------------------------------------------')